# Running cellpose locally with a GPU

Notebook version to run cellpose locally with GPU access on windows and Linux systems, the code cells that are going to be executed are indicated with [ ] at the left, run them with the play button or ctrl + enter

## Setting up your notebook

We are going to setup everything in order to run the cellpose analysis. 

Check CUDA version and GPU

In [1]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_22:08:44_Pacific_Standard_Time_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
Mon Mar 28 08:33:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.79       Driver Version: 511.79       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000    WDDM  | 00000000:73:00.0  On |                  Off |
| 33%   31C    P8    16W / 260W |    727MiB / 49152MiB |      4%      Default |
|                    

Now we are going to import the libraries and check GPU (the first time you import cellpose the models will download).

In [2]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import tifffile as tf
import skimage.io
import zarr

from urllib.parse import urlparse
from cellpose import models, core

use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)

# call logger_setup to have output of cellpose written
from cellpose.io import logger_setup
logger_setup();


C:\Users\Linux\.conda\envs\cellpose2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


>>> GPU activated? 1
2022-03-28 08:33:37,355 [INFO] WRITING LOG OUTPUT TO C:\Users\Linux\.cellpose\run.log


### Load your images

Here you can put the files that are going to be processed, just separate them with a ",".
As a guide you can copy the paths of several files in windows selecting them in the file explorer and *Shift + Rigth click* + *Copy as path*.
They are copied with windows filepath structure, in order to be listed, just add an r before the ".

In [3]:
# REPLACE FILES WITH YOUR IMAGE PATHS
files = ["D:\EHG\Explants\CYP17A1\C3-eh3931_cyclo25_explant-2_5-couptfii_6-insl3_7-cyp17a1-1s_12x_1z_blaze_C0.ome_Processed.aivia_Processed.aivia.tif64.zarr",
"D:\EHG\Explants\CYP17A1\C3-eh3931_cyclo25_explant-3_5-couptfii_6-insl3_7-cyp17a1-1s_12x_1z_blaze_C0.ome_Processed.aivia_Processed.aivia.tif65.zarr",
"D:\EHG\Explants\CYP17A1\C3-eh3931_cyclo25_explant-4_5-couptfii_6-insl3_7-cyp17a1-1s_12x_1z_blaze_C0.ome_Processed.aivia_Processed.aivia.tif29.zarr",
"D:\EHG\Explants\CYP17A1\C3-EH3931_DMSOEtOH_Explant-1_5-COUPTFII_6-INSL3_7-CYP17A1-1s_12x_1z_Blaze_C0.aivia_Processed.aivia.tif82.zarr",
"D:\EHG\Explants\CYP17A1\C3-EH3931_DMSOEtOH_Explant-2_5-COUPTFII_6-INSL3_7-CYP17A1-1s_12x_1z_Blaze_C0.ome_Processed.aivia_Processed.aivia.tif83.zarr",
"D:\EHG\Explants\CYP17A1\C3-EH3931_DMSOEtOH_Explant-3_5-COUPTFII_6-INSL3_7-CYP17A1-1s_12x_1z_Blaze_C0.ome_Processed.aivia_Processed.aivia.tif84.zarr",
"D:\EHG\Explants\CYP17A1\C3-EH3931_SAG7_Explant-1_5-COUPTFII_6-INSL3_7-CYP17A1-1s_12x_1z_Blaze_C0.ome_Processed.aivia_Processed.aivia.tif85.zarr",
"D:\EHG\Explants\CYP17A1\C3-EH3931_SAG7_Explant-2_5-COUPTFII_6-INSL3_7-CYP17A1-1s_12x_1z_Blaze_stitched_C0.ome_Processed.aivia_Processed.aivia.tif86.zarr",
"D:\EHG\Explants\CYP17A1\C3-EH3931_SAG7_Explant-3_5-COUPTFII_6-INSL3_7-CYP17A1-1s_12x_1z_Blaze_C0.ome_Processed.aivia_Processed.aivia.tif87.zarr"]
    
imgs = [zarr.load(f) for f in files]
#imgs = [tf.imread(f) for f in files]
nimg = len(imgs)

## Run cellpose 3D

There are two ways to run cellpose in 3D, this cells shows both, choose which one works best for you.

First way: computes flows from 2D slices and combines into 3D flows to create masks (Not so goog for me)

The only thing that have to be changed is the model type nad the chanels in [A,B]; A should be your cytoplasm channel and B your nuclei channel. If there is no nuclei channel, just left 0.

Also, remember to setup your estimated diameter in pixels.

In [ ]:
# DEFINE CELLPOSE MODEL
# model_type='cyto', 'nuclei', 'cyto2' or 'omni'
model = models.Cellpose(gpu=use_GPU, model_type='cyto2')

# * with 3D you have to set the diameter manually (no auto detect) *

### TWO WAYS TO RUN CELLPOSE IN 3D

# 1. computes flows from 2D slices and combines into 3D flows to create masks
print('running cellpose 2D slice flows => masks')
masks, flows, styles, diams = model.eval(imgs, channels=[0,0], diameter=15, do_3D=True)


Second way: computes masks in 2D slices and stitches masks in 3D based on mask overlap

Note: Stitching (with stitch_threshold > 0) can also be used to track cells over time.

In [ ]:
# DEFINE CELLPOSE MODEL
# model_type='cyto' or model_type='nuclei'
model = models.Cellpose(gpu=use_GPU, model_type='cyto2')

# 2. computes masks in 2D slices and stitches masks in 3D based on mask overlap
print('running cellpose 2D + stitching masks')
masks_stitched, flows_stitched, styles_stitched, _ = model.eval(imgs, channels=[0,0], diameter=20, do_3D=False, stitch_threshold=0.5)

2022-03-28 08:35:29,750 [INFO] ** TORCH CUDA version installed and working. **
2022-03-28 08:35:29,751 [INFO] >>>> using GPU
running cellpose 2D + stitching masks
2022-03-28 08:35:29,858 [INFO] ~~~ FINDING MASKS ~~~
2022-03-28 08:35:29,858 [INFO] 0%|          | 0/9 [00:00<?, ?it/s]
2022-03-28 08:35:29,858 [INFO] multi-stack tiff read in as having 256 planes 1 channels
2022-03-28 08:35:37,199 [INFO] 
2022-03-28 08:35:37,199 [INFO] 0%|          | 0/256 [00:00<?, ?it/s]
2022-03-28 08:35:37,214 [INFO] 
2022-03-28 08:35:52,842 [INFO] 
2022-03-28 08:35:52,842 [INFO] 0%|          | 1/256 [00:15<1:06:25, 15.63s/it]
2022-03-28 08:35:52,842 [INFO] 
2022-03-28 08:36:04,324 [INFO] 
2022-03-28 08:36:04,324 [INFO] 1%|          | 2/256 [00:27<55:49, 13.19s/it]
2022-03-28 08:36:04,324 [INFO] 
2022-03-28 08:36:15,886 [INFO] 
2022-03-28 08:36:15,886 [INFO] 1%|1         | 3/256 [00:38<52:28, 12.45s/it]
2022-03-28 08:36:15,886 [INFO] 
2022-03-28 08:36:27,860 [INFO] 
2022-03-28 08:36:27,860 [INFO] 2%|1    

### Save output

You will see the files saved in the folder where you have your original file.

You can choose to save the results as Tif files or Numpy arrays.

In [ ]:
#SAVE AS TIFF

from cellpose import io
io.save_masks(imgs, masks_stitched, flows_stitched, files, tif=True, save_flows=True)

In [ ]:
#SAVE AS NUMPY

from cellpose import io
io.masks_flows_to_seg(imgs, masks_stitched, flows_stitched, diams, files)

### Inspect the results in 3D using Napari

Load and open Napari viewer

In [ ]:
import napari
viewer = napari.Viewer()

Now you can drag ans drop your files to the viewer.

If you have few files, you can run the next cell to load them in Napari.